Analyse exploratoire

# Intro

In [1]:
# Import des bibliothèques
import os
os.chdir("..")
import pandas as pd
import numpy as np
import re
from utils.helper import parse_name, give_short_name

# Import des données
from data.cleaning import athlet, summer, winter
region = pd.read_csv("regions.csv", index_col=0)

In [2]:
print("'athlet':")
print(f"{athlet.shape[0]} lignes")
print(f"{athlet.shape[1]} colonnes")

'athlet':
286237 lignes
13 colonnes


In [3]:
athlet.head(3)

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,ShortName
index,,,,,,,,,,,,,
S000001,A Dijiang,M,24,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,Dijiang A
S000002,A Lamusi,M,23,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,Lamusi A
S000003,Gunnar Nielsen Aaby,M,24,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Gunnar AABY


# Athlètes : Nettoyage des données

## Extraire les noms

In [4]:
person = (pd.DataFrame(athlet
                        .groupby("Name").Event.agg('count')
                        .sort_values(ascending=False)
                        .reset_index()))
person.head()

,Name,Event
0,Robert Tait McKenzie,58
1,Heikki Ilmari Savolainen,39
2,"Joseph ""Josy"" Stoffel",38
3,Ioannis Theofilakis,36
4,Takashi Ono,33


In [5]:
person[person.Name.str.contains("Phelps")]

,Name,Event
15,"Michael Fred Phelps, II",30
4169,"Richard Lawson Phelps, Jr.",7
5084,"Robert Lawson Phelps, Sr.",6
7151,"Jaycie Lynn Phelps (-McClure, -Marus)",6
9352,Monica Kathleen Rutherford (-Phelps),5
31508,Mason Elliott Phelps,2
32809,Harlow Phelps Rothert,2
39453,"John Phelps ""Jack"" George",2
58195,Brian Eric Phelps,2
72932,Richard Charles Phelps,1


In [6]:
person["ShortName"] = person.Name.apply(give_short_name)
person["FirstName"] = person.Name.apply(lambda s: parse_name(s)["first"])
person["LastName"] = person.Name.apply(lambda s: parse_name(s)["last"].upper())
person = person[["Name", "FirstName", "LastName", "ShortName", "Event"]]
person.head()

,Name,FirstName,LastName,ShortName,Event
0,Robert Tait McKenzie,robert,MCKENZIE,Robert MCKENZIE,58
1,Heikki Ilmari Savolainen,heikki,SAVOLAINEN,Heikki SAVOLAINEN,39
2,"Joseph ""Josy"" Stoffel",joseph,STOFFEL,Joseph STOFFEL,38
3,Ioannis Theofilakis,ioannis,THEOFILAKIS,Ioannis THEOFILAKIS,36
4,Takashi Ono,takashi,ONO,Takashi ONO,33


In [7]:
short_names = (person
 .groupby("ShortName")
 .sum("Event")
 .sort_values(by="Event", ascending=False))
short_names.to_csv("../../draft/athlete_short_names.csv")
short_names.head(3)

,Event
ShortName,
Robert MCKENZIE,58
Gustaf CARLBERG,49
Heikki SAVOLAINEN,39


In [8]:
person[person["LastName"] == "SCHMIDT"]

,Name,FirstName,LastName,ShortName,Event
2662,Iohan Schmidt,iohan,SCHMIDT,Iohan SCHMIDT,8
7159,Magdalena Schmidt (-Jakob),magdalena,SCHMIDT,Magdalena SCHMIDT,6
8132,Oscar Daniel Bezerra Schmidt,oscar,SCHMIDT,Oscar SCHMIDT,5
10367,Florian Schmidt,florian,SCHMIDT,Florian SCHMIDT,4
15015,Ingrid Schmidt (-Naue),ingrid,SCHMIDT,Ingrid SCHMIDT,4
...,...,...,...,...,...
133023,Herbert Schmidt,herbert,SCHMIDT,Herbert SCHMIDT,1
133885,Heinrich Schmidt,heinrich,SCHMIDT,Heinrich SCHMIDT,1
134636,Ingrid Ulrike Schmidt (-Koppers),ingrid,SCHMIDT,Ingrid SCHMIDT,1
136868,Gustav Schmidt,gustav,SCHMIDT,Gustav SCHMIDT,1


In [9]:
names = (person
            .groupby(["ShortName", "FirstName", "LastName", "Name"])
            .sum("Event").reset_index()
            .sort_values(by="Event", ascending=False))
names.to_csv("../../draft/athlete_names.csv", index=None, chunksize=10000)
names

,ShortName,FirstName,LastName,Name,Event
116699,Robert MCKENZIE,robert,MCKENZIE,Robert Tait McKenzie,58
51535,Heikki SAVOLAINEN,heikki,SAVOLAINEN,Heikki Ilmari Savolainen,39
69622,Joseph STOFFEL,joseph,STOFFEL,"Joseph ""Josy"" Stoffel",38
56880,Ioannis THEOFILAKIS,ioannis,THEOFILAKIS,Ioannis Theofilakis,36
129728,Takashi ONO,takashi,ONO,Takashi Ono,33
...,...,...,...,...,...
61070,Janier HERNNDEZ,janier,HERNNDEZ,Janier Concepcin Hernndez,1
61069,Janie REED,janie,REED,REED Janie,1
61067,Janice TROMBLY,janice,TROMBLY,"Janice Yvonne ""Jan"" Trombly",1
61066,Janice TEIXEIRA,janice,TEIXEIRA,Janice Gil Teixeira,1


In [10]:
print(146361 - 141513)

4848


In [11]:
person[person["ShortName"] == "Zsuzsanna JAKABOS"]

,Name,FirstName,LastName,ShortName,Event
1476,"Zsuzsanna ""Zsu"" Jakabos",zsuzsanna,JAKABOS,Zsuzsanna JAKABOS,10
144651,JAKABOS Zsuzsanna,zsuzsanna,JAKABOS,Zsuzsanna JAKABOS,1


## Age : passer en entier

In [12]:
# On s'assure que tous les ages sont entiers
# avant de les convertir en "int"
athlet.Age.apply(lambda x: x.is_integer()).value_counts()

Age
True     276763
False      9474
Name: count, dtype: int64

In [13]:
# On s'assure que les ages non entiers
# ne correspondent qu'aux valeurs vides
athlet["intAge"] = athlet.Age.apply(lambda x: x.is_integer())
athlet.Age.isna().sum() == athlet[athlet["intAge"] == False].shape[0]

True

In [14]:
#assert len(athlete[athlete.Age == 0]) == 0
#athlete["Age"].fillna(0, inplace=True)
#athlete["Age"] = athlete.Age.astype("int64")
#athlete.replace(0, np.nan, inplace=True)
athlet["Age"] = athlet.Age.astype("Int64")
athlet.drop(columns=["intAge"], inplace=True)
athlet.head()

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
index,,,,,,,,,,,,
S000001,A Dijiang,M,24,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
S000002,A Lamusi,M,23,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
S000003,Gunnar Nielsen Aaby,M,24,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
S000004,Edgar Lindenau Aabye,M,34,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
S000005,"Cornelia ""Cor"" Aalten (-Strannood)",F,18,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN


In [15]:
athlet[athlet.Age.isna()]

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
index,,,,,,,,,,,,
S000086,Mohamed Jamshid Abadi,M,<NA>,Iran,IRI,1948 Summer,1948,Summer,London,Boxing,Boxing Men's Heavyweight,NaN
S000091,Georgi Abadzhiev,M,<NA>,Bulgaria,BUL,1924 Summer,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Individual",NaN
S000092,Georgi Abadzhiev,M,<NA>,Bulgaria,BUL,1924 Summer,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Team",NaN
S000101,Mohamed Abakkar,M,<NA>,Sudan,SUD,1972 Summer,1972,Summer,Munich,Boxing,Boxing Men's Flyweight,NaN
S000151,Sayed Fahmy Abaza,M,<NA>,Egypt,EGY,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
W047155,Boris Yakimov,M,<NA>,Soviet Union,URS,1956 Winter,1956,Winter,Cortina d'Ampezzo,Speed Skating,"Speed Skating Men's 5,000 metres",NaN
W047156,Boris Yakimov,M,<NA>,Soviet Union,URS,1956 Winter,1956,Winter,Cortina d'Ampezzo,Speed Skating,"Speed Skating Men's 10,000 metres",NaN
W047713,Luciano Zampatti,M,<NA>,Italy,ITA,1928 Winter,1928,Winter,Sankt Moritz,Ski Jumping,"Ski Jumping Men's Normal Hill, Individual",NaN


In [16]:
athlet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286237 entries, S000001 to W048564
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Name    286237 non-null  object
 1   Sex     286237 non-null  object
 2   Age     276763 non-null  Int64 
 3   Team    286237 non-null  object
 4   NOC     286237 non-null  object
 5   Games   286237 non-null  object
 6   Year    286237 non-null  int64 
 7   Season  286237 non-null  object
 8   City    286237 non-null  object
 9   Sport   286237 non-null  object
 10  Event   286237 non-null  object
 11  Medal   42232 non-null   object
dtypes: Int64(1), int64(1), object(10)
memory usage: 28.7+ MB


## Sex

In [4]:
print("Répartition des athlètes par genre :")
print(athlet["Sex"].value_counts())
print("en proportion :")
print(athlet["Sex"].value_counts(normalize=True))

Répartition des athlètes par genre :
Sex
M    204449
F     81788
Name: count, dtype: int64
en proportion :
Sex
M    0.714265
F    0.285735
Name: proportion, dtype: float64


In [5]:
# On vérifie qu'il n'y a pas de valeurs vides pour le genre
assert athlet[athlet["Sex"].isnull()].shape[0] == 0

In [8]:
# On vérifie si chaque athlète (regroupé sous le même nom court)
# a un unique genre identifié dans la base
athlet.groupby("ShortName").Sex.nunique().describe()

count    141513.000000
mean          1.000530
std           0.023015
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           2.000000
Name: Sex, dtype: float64

In [13]:
n_gender = athlet.groupby("ShortName").Sex.nunique().reset_index()
print(n_gender[n_gender["Sex"] == 2].count(), "pers. identifiées sous 2 genres")
n_gender[n_gender["Sex"] == 2].to_csv("../../draft/two_genders.csv", index=None)
n_gender[n_gender["Sex"] == 2]

ShortName    75
Sex          75
dtype: int64 pers. identifiées sous 2 genres


,ShortName,Sex
4486,Alex MORGAN,2
5118,Alexis THOMPSON,2
20715,Chan LUI,2
21672,Chen HONG,2
21690,Chen JING,2
...,...,...
140705,Zhang LI,2
140717,Zhang MIN,2
140719,Zhang NAN,2
140729,Zhang QING,2


N.B : certaines personnes sont peut-êtres identifiées sous un même nom court
car elles portent un prénom mixte et un nom courant.

*Ex* : la joueuse de football américaine Alex Morgan
a peut-être un homonyme masculin.

## Year

In [17]:
athlet["Year"] = athlet.Year.astype("Int64")
athlet["Year"].sample(n=10)

index
S133381    1972
S187497    2000
S223597    2020
W027020    1968
W010320    1976
S202964    1956
S113226    1980
S096882    1968
S081543    2000
S102519    1996
Name: Year, dtype: Int64

In [18]:
athlet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286237 entries, S000001 to W048564
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Name    286237 non-null  object
 1   Sex     286237 non-null  object
 2   Age     276763 non-null  Int64 
 3   Team    286237 non-null  object
 4   NOC     286237 non-null  object
 5   Games   286237 non-null  object
 6   Year    286237 non-null  Int64 
 7   Season  286237 non-null  object
 8   City    286237 non-null  object
 9   Sport   286237 non-null  object
 10  Event   286237 non-null  object
 11  Medal   42232 non-null   object
dtypes: Int64(2), object(10)
memory usage: 28.9+ MB


## Médailles

In [19]:
compet = (pd.DataFrame(athlet.groupby(["Games", "Event"]).Medal.count())
          .reset_index())
compet.sort_values(by="Medal", ascending=False)

,Games,Event,Medal
6228,2020 Summer,Men Team,335
6381,2020 Summer,Women Team,334
354,1908 Summer,Gymnastics Men's Team All-Around,94
599,1920 Summer,Gymnastics Men's Team All-Around,77
466,1912 Summer,"Gymnastics Men's Team All-Around, Swedish System",74
...,...,...,...
957,1928 Winter,"Speed Skating Men's 10,000 metres",0
1285,1948 Summer,"Art Competitions Mixed Painting, Unknown Event",0
1289,1948 Summer,"Art Competitions Mixed Sculpturing, Unknown Event",0
6466,2020 Summer,Women's Madison,0


In [20]:
compet.to_excel("../../draft/compet.ods")

In [21]:
compet.Medal.describe()

count    6498.000000
mean        6.499231
std        10.449955
min         0.000000
25%         3.000000
50%         3.000000
75%         6.000000
max       335.000000
Name: Medal, dtype: float64

In [22]:
athlet["Medal"].fillna(pd.NA, inplace=True)

In [23]:
athlet["Medal"] = athlet["Medal"].fillna("None")
athlet.Medal.value_counts()

Medal
None      244005
Gold       14172
Bronze     14162
Silver     13898
Name: count, dtype: int64

In [24]:
athlet["Medal"] = athlet.Medal.astype("category")
athlet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286237 entries, S000001 to W048564
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   Name    286237 non-null  object  
 1   Sex     286237 non-null  object  
 2   Age     276763 non-null  Int64   
 3   Team    286237 non-null  object  
 4   NOC     286237 non-null  object  
 5   Games   286237 non-null  object  
 6   Year    286237 non-null  Int64   
 7   Season  286237 non-null  object  
 8   City    286237 non-null  object  
 9   Sport   286237 non-null  object  
 10  Event   286237 non-null  object  
 11  Medal   286237 non-null  category
dtypes: Int64(2), category(1), object(9)
memory usage: 27.0+ MB


In [25]:
athlet

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
index,,,,,,,,,,,,
S000001,A Dijiang,M,24,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
S000002,A Lamusi,M,23,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
S000003,Gunnar Nielsen Aaby,M,24,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
S000004,Edgar Lindenau Aabye,M,34,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
S000005,"Cornelia ""Cor"" Aalten (-Strannood)",F,18,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,None
...,...,...,...,...,...,...,...,...,...,...,...,...
W048560,Andrzej ya,M,29,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,None
W048561,Piotr ya,M,27,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",None
W048562,Piotr ya,M,27,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",None


## NOC

In [26]:
print(athlet.NOC.nunique())

233


# Region

In [27]:
assert region.shape[0] == region.NOC.nunique()

In [28]:
region

,NOC,region,notes
0,EOR,Refugee,NaN
1,LBN,Lebanon,NaN
2,SGP,Singapore,NaN
3,ROC,Russia,NaN
4,AFG,Afghanistan,NaN
...,...,...,...
229,YEM,Yemen,NaN
230,YMD,Yemen,South Yemen
231,YUG,Serbia,Yugoslavia
232,ZAM,Zambia,NaN


In [29]:
region.loc[region["NOC"] == "ROT", "region"] = "Refugee"
region.loc[region["NOC"] == "TUV", "region"] = "Tuvalu"
region.loc[region["NOC"] == "UNK", "region"] = "Unknown"

In [30]:
region.columns = map(str.lower, region.columns)
region

,noc,region,notes
0,EOR,Refugee,NaN
1,LBN,Lebanon,NaN
2,SGP,Singapore,NaN
3,ROC,Russia,NaN
4,AFG,Afghanistan,NaN
...,...,...,...
229,YEM,Yemen,NaN
230,YMD,Yemen,South Yemen
231,YUG,Serbia,Yugoslavia
232,ZAM,Zambia,NaN


In [31]:
region[~region["notes"].isna()]

,noc,region,notes
5,AHO,Curacao,Netherlands Antilles
10,ANT,Antigua,Antigua and Barbuda
11,ANZ,Australia,Australasia
30,BOH,Czech Republic,Bohemia
55,CRT,Greece,Crete
92,HKG,China,Hong Kong
97,IOA,Individual Olympic Athletes,Individual Olympic Athletes
103,ISV,"Virgin Islands, US",Virgin Islands
147,NBO,Malaysia,North Borneo
151,NFL,Canada,Newfoundland


In [32]:
region[region["region"] == "Tuvalu"]

,noc,region,notes
212,TUV,Tuvalu,Tuvalu


In [33]:
for reg in list(region["region"].unique())[:25]:
    print(reg)

Refugee
Lebanon
Singapore
Russia
Afghanistan
Curacao
Albania
Algeria
Andorra
Angola
Antigua
Australia
Argentina
Armenia
Aruba
American Samoa
Austria
Azerbaijan
Bahamas
Bangladesh
Barbados
Burundi
Belgium
Benin
Bermuda
